In [1]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn import linear_model

from keras import *
import tensorflow as tf
from keras.models import Sequential
from keras.layers.core import Dense, Dropout
from tensorflow.keras import regularizers
from keras.optimizers import Adam, RMSprop
from keras.backend import clear_session


from sklearn.linear_model import Ridge, Lasso

from sklearn.neighbors import KNeighborsRegressor as knn
from numpy import arange

from google.colab import drive

drive.mount('/content/drive')


Mounted at /content/drive


In [1]:
train = pd.read_csv("/content/drive/MyDrive/AML project/train_manipulated.csv")

NameError: ignored

In [3]:
## preparazione train e validation set

y_train = train['trip_duration'].values.reshape(-1, 1)

X_train = train.drop(columns = ['trip_duration'])

#si separano le dummies dalle quantitative

X_train_to_stand = X_train[['passenger_count', 'distance']]

X_train_not_stand = X_train.drop(columns = ['passenger_count', 'distance']).values

In [4]:
scaler = StandardScaler()

X_train = scaler.fit_transform(X_train_to_stand)
y_train = scaler.fit(y_train).fit_transform(y_train)

In [5]:
X_train = np.concatenate((X_train, X_train_not_stand), axis = 1)

In [6]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.25,
                                                  random_state = 1234)

In [7]:
X_train.shape

(1080324, 41)

## Ridge & Lasso

In [ ]:
def tuner(x_train, y_train, x_validation,  y_validation, grid, model):

  dct = dict()

  if model == 'Ridge':

    for lmda in grid:

      model = Ridge(alpha= lmda)
      model.fit(x_train, y_train)
      y_pred = model.predict(x_validation)
      score = mean_squared_error(y_validation, y_pred)
      dct[lmda] = score
      
    return dct

  else:

    for lmda in grid:

      model = Lasso(alpha= lmda, max_iter=3000)
      model.fit(x_train, y_train)
      y_pred = model.predict(x_validation)
      score = mean_squared_error(y_validation, y_pred)
      dct[lmda] = score

    return dct


In [ ]:
grid = np.linspace(start=0.0001, stop=0.1, num=100)
model_r = 'Ridge'

ridge_grd = tuner(X_train, y_train, X_val, y_val, grid, model_r)
ridge_opt_value = min(ridge_grd, key=ridge_grd.get)
print(ridge_opt_value)

0.0001


In [ ]:
model_r = Ridge(alpha = ridge_opt_value)
model_r.fit(X_train, y_train)
y_pred = model_r.predict(X_val)
mse_r = mean_squared_error(y_val, y_pred)
r_quadro_r = r2_score(y_val, y_pred)

print(mse_r)
print(r_quadro_r)

0.37846407718464325
0.6249419724628476


### Lasso


In [ ]:
model = 'Lasso'

lss_grd = tuner(X_train, y_train, X_val, y_val, grid, model)
lasso_opt_value = min(lss_grd, key=lss_grd.get)
print(lasso_opt_value)

0.0001


In [ ]:
model_l = Lasso(alpha=lasso_opt_value, max_iter=3000)
model_l.fit(X_train, y_train)
y_pred1 = model_l.predict(X_val)
mse_l = mean_squared_error(y_val, y_pred1)
r_quadro_l = r2_score(y_val, y_pred1)

print(mse_r)
print(r_quadro_r)

0.37846407718464325
0.6249419724628476


## KNN

In [ ]:
rmse_val = []
for K in range(5):
  print(K)
  K = K+1
  model = knn(n_neighbors = K)

  model.fit(X_train, y_train)  #fit the model
  pred=model.predict(X_val) #make prediction on test set
  error = sqrt(mean_squared_error(y_val,pred)) #calculate rmse
  rmse_val.append(error) #store rmse values
  print('RMSE value for k= ' , K , 'is:', error)

In [ ]:
#plotting the rmse values against k values
curve = pd.DataFrame(rmse_val) #elbow curve 
curve.plot()

In [ ]:
model = knn(n_neighbors= knn_opt_value)
model.fit(x_train, y_train)
y_pred2 = model.predict(x_validation)
mse_k = mean_squared_error(y_val, y_pred2)
r_quadro_k = r2_score(y_val, y_pred2)

print(mse_k)
print(r_quadro_k)

In [ ]:
rmse_val = []
for K in range(5):
  print(K)
  K = K+1
  model = knn(n_neighbors = K)

  model.fit(X_train, y_train)  #fit the model
  pred=model.predict(X_val) #make prediction on test set
  error = sqrt(mean_squared_error(y_val,pred)) #calculate rmse
  rmse_val.append(error) #store rmse values
  print('RMSE value for k= ' , K , 'is:', error)

In [ ]:
model = knn(n_neighbors = K)

model.fit(X_train, y_train)  #fit the model

pred=model.predict(X_val)